In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def calculate_vorp(num_teams = 10, roster_size = 15, 
                   num_qb = 1, num_rb = 2, num_wr = 2, num_te = 1, 
                   num_flex = 1, num_sflex = 0, num_k = 1, num_def = 1, 
                   pp_pass = 0, pp_passyd = 0.04, pp_passtd = 4, pp_pick = -2, pp_fum = -2,
                   pp_rushyd = 0.1, pp_rushtd = 6, 
                   pp_rec = 0.5, pp_recyd = 0.1, pp_rectd = 6, te_premium = 0,
                   pp_fg = 3, pp_fg40 = 1, pp_fg50 = 2, pp_xp = 1, pp_missfg = -1, pp_missxp = -1,
                   pp_defint = 2, pp_deffum = 2, pp_deftd = 6, pp_defpa = -1, pp_defya = -0.1, pp_defsack = 1, pp_defsafe = 2, pp_defblock = 2,
                   ):
    # df from csv
    position_data = pd.read_csv('data/position_data.csv')
    defense_data = pd.read_csv('data/defense_data.csv')
    kicker_data = pd.read_csv('data/kicker_data.csv')
    
    # projected fantasy points for each player
    position_data['PTS'] = position_data['PASSYDS']*pp_passyd + position_data['PASSTD']*pp_passtd + position_data['INT']*pp_pick + position_data['FUM']*pp_fum + position_data['RUSHYDS']*pp_rushyd + position_data['RUSHTD']*pp_rushtd + position_data['REC']*pp_rec + position_data['RECYDS']*pp_recyd + position_data['RECTD']*pp_rectd
    # add in premium for TEs
    position_data.loc[position_data['POS'] == 'TE', 'PTS'] += position_data.loc[position_data['POS'] == 'TE', 'REC']*te_premium

    #TODO: add in kicker and defense points

    # replacement levels (currently just doing projected num starters, maybe should do projected num rostered)
    qb_replacement = num_teams * (num_qb + num_sflex)
    rb_replacement = num_teams * (num_rb + 0.5*num_flex)
    wr_replacement = num_teams * (num_wr + 0.5*num_flex)
    te_replacement = num_teams * (num_te + 0*num_flex)
    flex_replacement = num_teams * (num_rb + num_wr + num_te + num_flex)
    sflex_replacement = num_teams * (num_qb + num_rb + num_wr + num_te + num_flex + num_sflex)
    k_replacement = num_teams * num_k
    def_replacement = num_teams * num_def

    #subtract replacement level from each player
    position_data['VORP'] = 0
    position_data.loc[position_data['POS'] == 'QB', 'VORP'] = position_data.loc[position_data['POS'] == 'QB', 'PTS'] - position_data.loc[position_data['POS'] == 'QB', 'PTS'].nlargest(qb_replacement).min()
    position_data.loc[position_data['POS'] == 'RB', 'VORP'] = position_data.loc[position_data['POS'] == 'RB', 'PTS'] - position_data.loc[position_data['POS'] == 'RB', 'PTS'].nlargest(rb_replacement).min()
    position_data.loc[position_data['POS'] == 'WR', 'VORP'] = position_data.loc[position_data['POS'] == 'WR', 'PTS'] - position_data.loc[position_data['POS'] == 'WR', 'PTS'].nlargest(wr_replacement).min()
    position_data.loc[position_data['POS'] == 'TE', 'VORP'] = position_data.loc[position_data['POS'] == 'TE', 'PTS'] - position_data.loc[position_data['POS'] == 'TE', 'PTS'].nlargest(te_replacement).min()

    #flex is rb, wr, te
    position_data['VORP_FLEX'] = 0
    position_data.loc[position_data['POS'].isin(['RB', 'WR', 'TE']), 'VORP_FLEX'] = position_data.loc[position_data['POS'].isin(['RB', 'WR', 'TE']), 'PTS'] - position_data.loc[position_data['POS'].isin(['RB', 'WR', 'TE']), 'PTS'].nlargest(flex_replacement).min()

    #superflex is qb, rb, wr, te
    position_data['VORP_SFLEX'] = 0
    position_data.loc[position_data['POS'].isin(['QB', 'RB', 'WR', 'TE']), 'VORP_SFLEX'] = position_data.loc[position_data['POS'].isin(['QB', 'RB', 'WR', 'TE']), 'PTS'] - position_data.loc[position_data['POS'].isin(['QB', 'RB', 'WR', 'TE']), 'PTS'].nlargest(sflex_replacement).min()

    #kicker
    kicker_data['VORP'] = kicker_data['PTS'] - kicker_data['PTS'].nlargest(k_replacement).min()
    kicker_data['VORP_FLEX'] = 0
    kicker_data['VORP_SFLEX'] = 0

    #defense
    defense_data['VORP'] = defense_data['PTS'] - defense_data['PTS'].nlargest(def_replacement).min()
    defense_data['VORP_FLEX'] = 0
    defense_data['VORP_SFLEX'] = 0

    #merge all the dataframes only with name position and VORPS
    all_data = pd.concat([position_data[['NAME', 'POS', 'VORP', 'VORP_FLEX', 'VORP_SFLEX']], kicker_data[['NAME', 'POS', 'VORP', 'VORP_FLEX', 'VORP_SFLEX']], defense_data[['NAME', 'POS', 'VORP', 'VORP_FLEX', 'VORP_SFLEX']]])

    return all_data